In [14]:
# Copyright 2022 The Forta Foundation
 
import requests
import pandas as pd
import json
import dotenv
import os

# Load environment variables
dotenv.load_dotenv()

True

In [15]:
def execute_query(sql: str) -> pd.DataFrame:
        url = "https://q.luabase.com/run"
        payload = {"block":
                   {"details": {
                    "sql": sql,
                    }
                    },
                   "api_key": os.environ.get('LUABASE_API_KEY')
                   }

        headers = {"content-type": "application/json"}
        response = requests.request("POST", url, json=payload, headers=headers, timeout=600)  # 10min timeout
        
        data = response.json()
        
        print(data)
        return pd.DataFrame(data["data"])

This notebook allows to backtest the attack detector V2 against a set of alerts. Independent of the threshold set within the bot, the log will be analyzed, so the relevant alerts, their anomaly score, overall anomaly score, and any FP filtering mitigations are shown.

Since there is a command line character limit, the local persistence mechanism needs to be enabled to persist state across executions (by setting local node property in constants.py to 1)

In [16]:
start_date = "2022-12-31" #when the attack detector first started to operate

# attacker_addresses = "0x14d8Ada7A0BA91f59Dc0Cb97C8F44F1d177c2195".lower()
# chain = "ethereum" #ethereum or binance 
# name = "TomInu"
# end_date = "2023-01-26" #should be the date of the attack


# attacker_addresses = "0x69f0EdC352eCffC4EF49516c9b20eA88B3E947cb".lower()
# chain = "ethereum" #ethereum or binance 
# name = "BlueCluesInu"
# end_date = "2023-01-26" #should be the date of the attack

attacker_addresses = "0x88a2386e7ec97ad1e7a72176a66b6d0711ae3527".lower()
chain = "ethereum" #ethereum or binance 
name = "Quaternion"
end_date = "2023-01-20" #should be the date of the attack


# attacker_addresses = "0xf3a465c9fa6663ff50794c698f600faa4b05c777,0x2ffc59d32a524611bb891cab759112a51f9e33c0" #comma separated list of attacker addresses (note, while all alerts from those addresses are pushed to the attack detector, one should check the clustering bot to see whether they are clustered together)
# chain = "ethereum" #ethereum or binance 
# name = "ANKR"
# end_date = "2022-12-04" #should be the date of the attack

# start_date = "2023-01-10" #when the attack detector first started to operate
# attacker_addresses = "0x9d0163e76bbcf776001e639d65f573949a53ab03"
# chain = "ethereum" #ethereum or binance 
# name = "LendHub"
# end_date = "2023-01-14" #should be the date of the attack

# attacker_addresses = "0x0348d20b74ddc0ac9bfc3626e06d30bb6fac213b"
# chain = "ethereum" #ethereum or binance 
# name = "JayExploiter"
# end_date = "2022-12-31" #should be the date of the attack

# attacker_addresses = "0x001B91c794dFEecf00124D3F9525DD32870B6ee9" #comma separated list of attacker addresses (note, while all alerts from those addresses are pushed to the attack detector, one should check the clustering bot to see whether they are clustered together)
# chain = "ethereum" #ethereum or binance 
# name = "CryptoRubric"
# end_date = "2022-12-28" #should be the date of the attack

# attacker_addresses = "0x3bdF01ed32F07e8e843163b5d478d4502F5743CD,0x25fDe76A52D01c83E31d2d3D5e1d2011ff103c56,0xBeAdeDBABED6A353c9cAa4894Aa7E5F883e32967" #comma separated list of attacker addresses (note, while all alerts from those addresses are pushed to the attack detector, one should check the clustering bot to see whether they are clustered together)
# chain = "ethereum" #ethereum or binance 
# name = "ElasticSwap"
# start_date = "2022-11-30" #when the attack detector first started to operate
# end_date = "2022-12-15" #should be the date of the attack

# attacker_addresses = "0x86Aa1c46f2Ae35ba1B228dC69fB726813D95b597,0x3c4e5b099f3c02122079d124138377e1b9048629" #comma separated list of attacker addresses (note, while all alerts from those addresses are pushed to the attack detector, one should check the clustering bot to see whether they are clustered together)
# chain = "binance" #ethereum or binance 
# name = "Nimbus"
# end_date = "2022-12-15" #should be the date of the attack

# attacker_addresses = "0xF4FD2EbE7196c8E99E88bcc4Aef69dda0e493B8F" #comma separated list of attacker addresses (note, while all alerts from those addresses are pushed to the attack detector, one should check the clustering bot to see whether they are clustered together)
# chain = "binance" #ethereum or binance 
# name = "BlackGold"
# end_date = "2022-12-15" #should be the date of the attack

# attacker_addresses = "0x286E09932B8D096cbA3423d12965042736b8F850" #comma separated list of attacker addresses (note, while all alerts from those addresses are pushed to the attack detector, one should check the clustering bot to see whether they are clustered together)
# chain = "binance" #ethereum or binance 
# name = "aespool"
# end_date = "2022-12-17" #should be the date of the attack

# attacker_addresses = "0xf71708C59be7e32B9FF2aa174F07311869C6bf0c" #comma separated list of attacker addresses (note, while all alerts from those addresses are pushed to the attack detector, one should check the clustering bot to see whether they are clustered together)
# chain = "binance" #ethereum or binance 
# name = "optionroom"
# end_date = "2022-12-17" #should be the date of the attack




In [17]:
# #clustering
# for attacker_address in attacker_addresses.split(","):
#     sql = f"SELECT metadata FROM forta.{chain}_alerts WHERE bot_id = '0xd3061db4662d5b3406b52b20f34234e462d2c275b99414d76dc644e2486be3e9' and CAST(substring(block_timestamp,1,19) as datetime)  >= '2022-10-01T00:00:00' AND CAST(substring(block_timestamp,1,19)  as datetime)  <= '{end_date}T00:00:00' and  lower(arrayStringConcat(metadata)) like '%{attacker_address.lower()}%'"
#     print(sql)
#     df = execute_query(sql)
#     print(df)

In [18]:
alert_hashes_df = pd.DataFrame(columns=["hash","createdAt"])
for attacker_address in attacker_addresses.split(","):
    url = 'https://api.forta.network/graphql'
    query = '''query exampleQuery {
        # first 5 alerts
        alerts(
            input: { first: 2000 , addresses: ["'''+attacker_address+'''"],
            blockDateRange: { startDate: "'''+start_date+'''", endDate: "'''+end_date+'''" }
            }
        ) {
            pageInfo {
            hasNextPage
            endCursor {
                alertId
                blockNumber
            }
            }
            alerts {
            hash
            createdAt
            
            }
        }
        }
    '''
    r = requests.post(url, json={'query': query})
    print(r.text)
    json_data = json.loads(r.text)
    df = pd.DataFrame(json_data['data']['alerts']['alerts'])
    print(df)
    alert_hashes_df = pd.concat([alert_hashes_df, df])

len(alert_hashes_df)

{"data":{"alerts":{"pageInfo":{"hasNextPage":false,"endCursor":{"alertId":"","blockNumber":0}},"alerts":[{"hash":"0x5e5cb21535537e4c7e37fa1f7f3f7c63e8d7b34d83e3d71f00e7dcd01b2c4bbb","createdAt":"2023-01-19T00:17:50.382196236Z"},{"hash":"0x1d9dddde8b1faa3db34e676e56dc9335b5d22c01e17f621b3ad40f21c24d5714","createdAt":"2023-01-20T15:21:52.536242328Z"},{"hash":"0x1972e0cd35aa24b33d402d1302684f1d1ca74f053fe31bb90f60d31e543cf686","createdAt":"2023-01-20T15:21:58.852555776Z"},{"hash":"0x3816b2a9357e014fee3381587e61adc23d2e41c412dbd439b21a2012ca62342c","createdAt":"2023-01-20T04:24:08.9411493Z"},{"hash":"0x57a52b154ceaae14410f0e41a3fd9c09ba9facdd14ec48d443a697ef2e89d8f0","createdAt":"2023-01-19T18:51:48.730141103Z"},{"hash":"0xfc7f1f2988153ad3d6ad79e5825644e5f5719f418070e0365a411997861f6df4","createdAt":"2023-01-19T07:55:52.802919224Z"},{"hash":"0x89c73f52bdaa044345f3080685066fc5f7318cf19bc2e6aa58c77cbea0936a53","createdAt":"2023-01-19T07:41:00.598765726Z"},{"hash":"0x66e760434dfdd859a27a6db0c

270

In [19]:
alert_hashes_df[alert_hashes_df["hash"]=='0x4c77f000d131f7180b7f14b69a1f6ac49446ea925adfe3a07a26b0431e91acfe']

,hash,createdAt


In [20]:
len(alert_hashes_df)

270

In [21]:
#remove all _key files (L2 cache)
!echo 'output.txt' > {name}.txt

#run npm run to get and route to output.txt
count = 0
alert_hashes = ""
for alert_hash in alert_hashes_df["hash"]:
    count += 1
    alert_hashes += f",{alert_hash}"
    if count % 90 == 0:
        !npm run alert {alert_hashes} >> {name}.txt 2>> {name}.txt
        alert_hashes = ""
if count % 90 != 0:
    !npm run alert {alert_hashes} >> {name}.txt 2>> {name}.txt



In [22]:
!grep 'ERROR' {name}.txt


2023-02-03 16:13:32,448 - root - ERROR - Exception in get_etherscan_label 'NoneType' object is not subscriptable
2023-02-03 16:13:32,505 - root - ERROR - Exception in get_etherscan_label 'NoneType' object is not subscriptable
2023-02-03 16:13:33,512 - root - ERROR - Exception in get_etherscan_label 'NoneType' object is not subscriptable
2023-02-03 16:13:35,788 - root - ERROR - Exception in get_etherscan_label 'NoneType' object is not subscriptable
2023-02-03 16:13:39,007 - root - ERROR - Exception in get_etherscan_label 'NoneType' object is not subscriptable
2023-02-03 16:13:41,303 - root - ERROR - Exception in get_etherscan_label 'NoneType' object is not subscriptable
2023-02-03 16:13:42,673 - root - ERROR - Exception in get_etherscan_label 'NoneType' object is not subscriptable
2023-02-03 16:13:43,305 - root - ERROR - Exception in get_etherscan_label 'NoneType' object is not subscriptable
2023-02-03 16:13:45,269 - root - ERROR - Exception in get_etherscan_label 'NoneType' object is n

In [23]:
!grep 'WARN' {name}.txt

2023-02-03 16:13:30,393 - root - WARNING - alert 0x89c73f52bdaa044345f3080685066fc5f7318cf19bc2e6aa58c77cbea0936a53 - no anomaly_score in metadata found: {'addressContainedInCreatedContract1': '0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640', 'addressContainedInCreatedContract2': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', 'addressContainedInCreatedContract3': '0x4c5813b8c6FbbAC76CAA148aAf8910f236B56fDF', 'addressContainedInCreatedContract4': '0xFEeCbA581F7b69245EB059c19FdbA56779A4c341', 'addressContainedInCreatedContract5': '0xBB82c3DEF4E9631586d1C57F8Be5328EB42EbF5b'}. Treating as anomaly_score of 1.0.
2023-02-03 16:13:38,760 - root - WARNING - alert 0x678f0dc99e9f6100d43681de3b36bcb3e12334857214e734ceae221307090013 - no anomaly_score in metadata found: {'addressContainedInCreatedContract1': '0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640', 'addressContainedInCreatedContract2': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', 'addressContainedInCreatedContract3': '0x4c5813b8c6FbbAC76CAA148aAf

In [24]:
!grep 'adding cluster mapping' {name}.txt

In [25]:
!grep 'adding FP mitigation cluster' {name}.txt

2023-02-03 16:16:25,746 - root - INFO - alert 0xfcbb38ba4726ba16256170cf1cc8d24e555c24a7d8fce3dc00956759ce4e6d29 adding FP mitigation cluster: 0x88a2386e7ec97ad1e7a72176a66b6d0711ae3527. FP mitigation clusters size now: 0


In [26]:
#read output.txt and answer questions around
!grep 'anomaly' {name}.txt


#anomaly scores
#FP filtering


2023-02-03 16:13:28,431 - root - INFO - alert 0xfc7f1f2988153ad3d6ad79e5825644e5f5719f418070e0365a411997861f6df4 0x3858be37e155f84e8e0d6212db1b47d4e83b1d41e8a2bebecb902651ed1125d6 NETHFORTA-1 Exploitation: 0x88a2386e7ec97ad1e7a72176a66b6d0711ae3527 anomaly score of 0.48
2023-02-03 16:13:30,393 - root - WARNING - alert 0x89c73f52bdaa044345f3080685066fc5f7318cf19bc2e6aa58c77cbea0936a53 - no anomaly_score in metadata found: {'addressContainedInCreatedContract1': '0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640', 'addressContainedInCreatedContract2': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', 'addressContainedInCreatedContract3': '0x4c5813b8c6FbbAC76CAA148aAf8910f236B56fDF', 'addressContainedInCreatedContract4': '0xFEeCbA581F7b69245EB059c19FdbA56779A4c341', 'addressContainedInCreatedContract5': '0xBB82c3DEF4E9631586d1C57F8Be5328EB42EbF5b'}. Treating as anomaly_score of 1.0.
2023-02-03 16:13:30,393 - root - INFO - alert 0x89c73f52bdaa044345f3080685066fc5f7318cf19bc2e6aa58c77cbea0936a53 0x457a